# Google Gemini SDK Introduction

This notebook demonstrates how to use the Google Gemini AI SDK to interact with the Gemini model in both text and audio modes.

## Setup
First, we'll install the required package and initialize the client with our API key.

In [1]:
!pip install -U -q google-genai

In [2]:
from google import genai
client = genai.Client(vertexai=False, http_options= {'api_version': 'v1alpha'}, api_key='<YOUR_API_KEY>')

## Text Interaction Example

Below we'll demonstrate how to have a text conversation with Gemini. The code:
1. Sets up a configuration for text responses
2. Opens an async connection to the model
3. Sends a message and receives the response in chunks
4. Prints each chunk of the response as it arrives

In [3]:
MODEL = "gemini-2.0-flash-exp"

In [4]:
config={
    "generation_config": {"response_modalities": ["TEXT"]}}

async with client.aio.live.connect(model=MODEL, config=config) as session:
  message = "Hello? Gemini are you there?"
  print("> ", message, "\n")
  await session.send(message, end_of_turn=True)

  # For text responses, When the model's turn is complete it breaks out of the loop.
  turn = session.receive()
  async for chunk in turn:
    if chunk.text is not None:
      print(f'- {chunk.text}')

>  Hello? Gemini are you there? 

- Yes, I'
- m here! How can I help you today?



## Audio Generation Example

Now we'll see how to generate audio responses from Gemini. This section:
1. Creates a wave file handler to save the audio
2. Configures the model for audio output
3. Sends a text prompt and receives audio data
4. Saves the audio chunks and plays them in the notebook

Note: Make sure your browser's audio is enabled to hear the responses.

In [5]:
import contextlib
import wave


@contextlib.contextmanager
def wave_file(filename, channels=1, rate=24000, sample_width=2):
    with wave.open(filename, "wb") as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(sample_width)
        wf.setframerate(rate)
        yield wf

In [6]:
from IPython.display import display, Audio

config={
    "generation_config": {"response_modalities": ["AUDIO"]}}

async with client.aio.live.connect(model=MODEL, config=config) as session:
  file_name = 'audio.wav'
  with wave_file(file_name) as wav:
    message = "Hello? Gemini are you there?"
    print("> ", message, "\n")
    await session.send(message, end_of_turn=True)

    first = True
    async for response in session.receive():
      if response.data is not None:
        model_turn = response.server_content.model_turn
        if first:
          print(model_turn.parts[0].inline_data.mime_type)
          first = False
        print('.', end='.')
        wav.writeframes(response.data)


display(Audio(file_name, autoplay=True))

>  Hello? Gemini are you there? 

audio/pcm;rate=24000
..............................